### 파킨슨병 데이터
- 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상태 status(1: 파킨슨병 진단, 0: 파킨슨병 아님)로 추가한 테이블
- (data/parkinsons.csv)
1. 파킨슨 병을 예측하는 모델로 로지스틱 회귀모형을 적용하여 생성
2. 파킨슨병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정
3. 파킨슨 병을 진단하는 기준를 함수로 생성하여(매개변수명 = threshold, 함수명 = cutoff)을 0.5로 했을 때와 0.8로 했을 때 F1-스코어를 비교
    - 분석 조건
        - 필요 없는 컬럼 name을 삭제
        - 데이터의 정규화는 min-max 스케일러 사용
        - 로지스틱 회귀를 위한 상수항 추가
        - status는 카테고리 타입으로 변환
        - 트레이닝셋과 테스트셋 비율은 9:1
        - 모델은 로지스틱 회귀분석 사용
        - 모델의 최적화 방법론은 "bfgs" 사용

In [16]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [40]:
df = pd.read_csv('./data/1일차_data/parkinsons.csv')
df

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


> 1.

In [41]:
# name컬럼 삭제
df = df.drop('name', axis=1)

In [42]:
df.head(1)

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.0037,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654


In [43]:
# 카테고리 타입으로 변환
df['status'] = df['status'].astype('category')

In [47]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   MDVP:Fo(Hz)       195 non-null    float64 
 1   MDVP:Fhi(Hz)      195 non-null    float64 
 2   MDVP:Flo(Hz)      195 non-null    float64 
 3   MDVP:Jitter(%)    195 non-null    float64 
 4   MDVP:Jitter(Abs)  195 non-null    float64 
 5   MDVP:RAP          195 non-null    float64 
 6   MDVP:PPQ          195 non-null    float64 
 7   Jitter:DDP        195 non-null    float64 
 8   MDVP:Shimmer      195 non-null    float64 
 9   MDVP:Shimmer(dB)  195 non-null    float64 
 10  Shimmer:APQ3      195 non-null    float64 
 11  Shimmer:APQ5      195 non-null    float64 
 12  MDVP:APQ          195 non-null    float64 
 13  Shimmer:DDA       195 non-null    float64 
 14  NHR               195 non-null    float64 
 15  HNR               195 non-null    float64 
 16  status            195 non-

In [48]:
# 상수항 추가
import statsmodels.api as sm

df = sm.add_constant(df,has_constant = "add")

In [50]:
# 데이터 분할
from sklearn.model_selection import train_test_split

feature_columns = df.columns.difference(['status'])

# feature_columns = list(df_sc.columns.difference(['status']))

x = df[feature_columns]
y = df['status']

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, train_size=0.9, random_state= 1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(175, 23) (20, 23) (175,) (20,)


In [51]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

Mmscaler = MinMaxScaler()

x_train_mm = Mmscaler.fit_transform(x_train)
x_test_mm = Mmscaler.fit_transform(x_test)

In [53]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')

lr.fit(x_train_mm, y_train)

LogisticRegression()

In [54]:
lr_preds = lr.predict(x_test_mm)
lr_preds


array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1],
      dtype=int64)

> 2.

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

cross_val_score(lr, x_train_mm, y_train, cv=5)
lr.score(x_test_mm, y_test)

feature_importance_lr = pd.DataFrame(zip(x.columns.values, lr.coef_.ravel()))
feature_importance_lr.columns = ['feature', 'coef']
feature_importance_lr.sort_values("coef", ascending=False, inplace=True)

feature_importance_lr.head(3)

,feature,coef
21,spread1,1.816588
15,PPE,1.537989
0,D2,1.443381


> 3.

In [55]:
def cutoff(y, threshold) :
    Y = y.copy()
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

In [57]:
from sklearn.metrics import f1_score

pred_Y_5 = cutoff(lr_preds, 0.5)
pred_Y_8 = cutoff(lr_preds, 0.8)

print(f'threshold = 0.5 일때 : {f1_score(y_test, pred_Y_5):.3f}')
print(f'threshold = 0.8 일때 : {f1_score(y_test, pred_Y_8):.3f}')


threshold = 0.5 일때 : 0.839
threshold = 0.8 일때 : 0.839
